# Preprocess 

In [5]:
%pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from collections import Counter
import spacy

In [ ]:
data = pd.read_csv("../../data/processed/train_limpieza_v1.csv")

In [ ]:
nlp = spacy.load('en_core_web_sm')

ModuleNotFoundError: No module named 'spacy'

## Statement

### Tokenizar, estities y stopwords por statemen 

In [ ]:
# Función para procesar el statement y extraer las métricas
def process_statement(statement):
    # Procesar el texto con spaCy
    doc = nlp(statement.lower()) #Convertimos todos los tokens a minusculas
    # Número de tokens
    num_tokens = len(doc)
    # Número de oraciones
    num_sentences = len(list(doc.sents))
    # Extraer POS tags, lemas y profundidad (depth)
    pos_info = []
    for token in doc:
        pos_info.append({
            'lemma': token.lemma_,
            'pos': token.pos_,
            'tag': token.tag_,
            'depth': token.dep_,
        })
    
    # Frecuencia de POS tags, lemas, etc.
    pos_freq = Counter([token['pos'] for token in pos_info])  # Frecuencia de POS tags
    lemma_freq = Counter([token['lemma'] for token in pos_info])  # Frecuencia de lemas
    tag_freq = Counter([token['tag'] for token in pos_info])  # Frecuencia de tags

    # Extraer las entidades, tipos y explicaciones
    entidades = []
    tipos = []
    explicaciones = []
    
    for ent in doc.ents:
        entidades.append(ent.text)
        tipos.append(ent.label_)
        explicaciones.append(spacy.explain(ent.label_))
    
    # ===================== Entities =======================================================
    # Crear un DataFrame para las entidades
    entities_df = pd.DataFrame({
        "Entidad": entidades,
        "Tipo de Entidad": tipos,
        "Explicación": explicaciones
    })

    # ===================== Stopwords =======================================================
    # Extracción de stopwords
    stopwords = nlp.Defaults.stop_words
    stopwords_in_statement = [token.text for token in doc if token.is_stop]
    
    # Filtrar los tokens que no son stopwords
    tokens_without_stopwords = [token for token in doc if not token.is_stop and not token.is_digit and not token.is_punct and token.text != '-' and token.text != '_']
    
    # Generar el statement sin stopwords
    statement_without_stopwords = ' '.join([token.text for token in tokens_without_stopwords])
    
    # Calcular las métricas para el statement sin stopwords
    num_tokens_without_stopwords = len(tokens_without_stopwords)
    
    # Extraer POS tags, lemas y profundidad para el statement sin stopwords
    pos_info_without_stopwords = []
    for token in tokens_without_stopwords:
        pos_info_without_stopwords.append({
            'lemma': token.lemma_,
            'pos': token.pos_,
            'tag': token.tag_,
            'depth': token.dep_,
        })
    
    # Frecuencia de POS tags, lemas, etc. para el statement sin stopwords
    pos_freq_without_stopwords = Counter([token['pos'] for token in pos_info_without_stopwords])  # Frecuencia de POS tags
    lemma_freq_without_stopwords = Counter([token['lemma'] for token in pos_info_without_stopwords])  # Frecuencia de lemas
    tag_freq_without_stopwords = Counter([token['tag'] for token in pos_info_without_stopwords])  # Frecuencia de tags
    
    # Retornar los resultados
    return (
        ' '.join([token.text for token in doc]), 
        num_tokens, 
        num_sentences, 
        pos_info, 
        pos_freq, 
        lemma_freq, 
        tag_freq, 
        entities_df, 
        stopwords_in_statement, 
        statement_without_stopwords, 
        num_tokens_without_stopwords, 
        pos_info_without_stopwords, 
        pos_freq_without_stopwords, 
        lemma_freq_without_stopwords, 
        tag_freq_without_stopwords
    )

In [ ]:
data[['statement_tokens', 'num_tokens', 'num_sentences', 'pos_info', 'pos_freq', 'lemma_freq', 'tag_freq',
    'entities', 'stopwords', 'statement_tokens_without_stopwords', 'num_tokens_without_stopwords', 
    'pos_info_without_stopwords', 'pos_freq_without_stopwords', 'lemma_freq_without_stopwords', 
    'tag_freq_without_stopwords']] = data['statement'].apply(lambda x: pd.Series(process_statement(x)))

## Subject

In [ ]:
def preprocess_subject(subject):
    # Convertir todo a minúsculas
    subject = subject.lower()

    # Eliminar puntuaciones y caracteres no alfabéticos, excepto guiones
    # Permitir guiones porque los usamos para unificar categorías como "state-financial"
    subject = ''.join([char if char.isalnum() or char in ['-', ','] else ' ' for char in subject])

    # Dividir por delimitadores (coma, guion, espacio)
    subject_tokens = [token.strip() for token in subject.split(',')]

    # Filtrar tokens si es necesario (ej. eliminar stopwords)
    stopwords = nlp.Defaults.stop_words
    subject_tokens = [token for token in subject_tokens if token not in stopwords]

    # Retornar los resultados
    return subject_tokens

# Aplicar el preprocesamiento a la columna 'subject'
data['processed_subject'] = data['subject'].apply(preprocess_subject)

In [ ]:

# Inicializar un contador
subject_counter = Counter()

# Contar la frecuencia de cada término en 'processed_subject'
for subjects in data['processed_subject']:
    subject_counter.update(subjects)

for token, count in subject_counter.items():
    print(f'{token}: {count}')

## Speaker

In [ ]:
def extract_entities_from_speaker(speaker):
    # Procesar el texto con spaCy
    doc = nlp(speaker)
    
    # Listas para almacenar entidades, tipos y explicaciones
    entidades = []
    tipos = []
    
    # Iterar sobre las entidades reconocidas
    for ent in doc.ents:
        entidades.append(ent.text)
        tipos.append(ent.label_)
    
    # Retornar las listas de entidades, tipos y explicaciones
    return entidades, tipos

# Aplicar la función para extraer las entidades de cada 'speaker'
data[['speaker_entities', 'speaker_type']] = data['speaker'].apply(lambda x: pd.Series(extract_entities_from_speaker(x)))
